### ベースラインモデルの構築を目指す

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
import pickle

### データの前処理

In [3]:
np.random.seed(2018)

# データの呼び出し
trn = pd.read_csv('./dataset/train_ver2.csv')
tst = pd.read_csv('./dataset/test_ver2.csv')

# 商品の変数を別途保存
prods = trn.columns[24:].tolist()

# 商品変数の欠損値をあらかじめ0に代替
trn[prods] = trn[prods].fillna(0.0).astype(np.int8)

# 24個の商品を1つも保有していない顧客のデータを除去
# チルダ(~)は否定の演算子
no_product = trn[prods].sum(axis=1) == 0
trn = trn[~no_product]

# 訓練データとテストデータを統合、テストデータにない商品変数は0で埋める
for col in trn.columns[24:]:
    tst[col] = 0
df = pd.concat([trn, tst], axis=0)

C:\Users\mhira\AppData\Local\Temp\ipykernel_25112\2408484113.py:4: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  trn = pd.read_csv('./dataset/train_ver2.csv')
C:\Users\mhira\AppData\Local\Temp\ipykernel_25112\2408484113.py:5: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  tst = pd.read_csv('./dataset/test_ver2.csv')


In [4]:
# 学習に使用する特徴量のリスト
features = []

# カテゴリ変数を、.factorize()関数に通して、label encoding
categorical_cols = [
    'ind_empleado', 'pais_residencia', 'sexo', 'tiprel_1mes', 
    'indresi', 'indext', 'conyuemp', 'canal_entrada', 
    'indfall', 'tipodom', 'nomprov', 'segmento'
]
for col in categorical_cols:
    df[col], _ = df[col].factorize()
features += categorical_cols

In [5]:
# 数値型変数の特異値と欠損値を -99 に代替し、整数型に変換
# 多分ここら辺の変換するべき値はやりながら見つけていけばいいと思う
df['age'].replace(' NA', -1, inplace=True)
df['age'] = df['age'].astype(np.int8)

df['antiguedad'].replace(' NA', -1, inplace=True)
df['antiguedad'].replace('     NA', -1, inplace=True)
df['antiguedad'] = df['antiguedad'].astype(np.int8)

df['renta'].replace(' NA', -1, inplace=True)
df['renta'].replace('         NA', -1, inplace=True)
df['renta'].fillna(-1, inplace=True)
df['renta'] = df['renta'].astype(float).astype(np.int8)

df['indrel_1mes'].replace('P', 5, inplace=True)
df['indrel_1mes'].fillna(-1, inplace=True)
df['indrel_1mes'] = df['indrel_1mes'].astype(float).astype(np.int8)

# 学習に使用する数値型変換を featuresに追加
features += ['age', 'antiguedad', 'renta', 'ind_nuevo', 'indrel', 'indrel_1mes', 'ind_actividad_cliente']


### 特徴量エンジニアリング

In [6]:
# 2つの日付変数から年度と月の情報を抽出
df['fecha_alta_month'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['fecha_alta_year'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['fecha_alta_month', 'fecha_alta_year']

df['ult_fec_cli_1t_month'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['ult_fec_cli_1t_year'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['ult_fec_cli_1t_month', 'ult_fec_cli_1t_year']


In [7]:
# それ以外の変数の欠損値を全て -1に代替
df.fillna(-1, inplace=True)

# lag-1データを生成
# 日付を数字に変換する変数（2015-01-28:1, ... , 2016-06-28:18）
def date_to_int(str_date):
    Y, M, D = [int(a) for a in str_date.strip().split("-")]
    int_date = (Y-2015)*12+M
    return int_date

# 日付を数字に変換し、int_dateに保存
df['int_date'] = df['fecha_dato'].map(date_to_int).astype(np.int8)

# データをコピーし、int_dateの日付に１を加えてlagを生成。識別番号と日付以外のカラム名に_prevを追加
# lagデータは時系列データにおける過去データのこと
df_lag = df.copy()
df_lag['int_date'] += 1
df_lag.columns = [col+'_prev' if col not in ['ncodpers', 'int_date'] else col for col in df.columns]

# 原本データとlagデータを識別番号と日付を基準として合わせる
# lagデータの日付は1だけ押されているので、前の月の商品情報が挿入される
df_trn = df.merge(df_lag, on=['ncodpers', 'int_date'], how='left')

# メモリ解放
del df, df_lag

# 前の月の商品情報が存在しない場合に備え、0に代替
for prod in prods:
    prev = prod+'_prev'
    df_trn[prev].fillna(0, inplace=True)
df_trn.fillna(-1, inplace=True)

# lag-1変数を追加
features += [feature+'_prev' for feature in features]
features += [prod+'_prev' for prod in prods]

###
### Baselineモデル以後は、多様な特徴量エンジニアリングを追加
###


### 機械学習モデルの学習
- 今回は2015-01-28~2016-05-28の1年5ヶ月分のデータから、2016-06-28、未来のデータを推測する
- こういうときは、最新の2016-05-28のデータに対して、交差検証を行う
- モデルを簡素化するために、訓練データは2016-01-28~2016-04-28の4ヶ月分とする

In [8]:
# 学習のため、データを訓練、検証用に分離
# 学習には、2016-01-28~2016-04-28のデータだけを使用し、検証には2016-05-28のデータを使用
use_dates = ['2016-01-28', '2016-02-28', '2016-03-28', '2016-04-28', '2016-05-28']
trn = df_trn[df_trn['fecha_dato'].isin(use_dates)]
tst = df_trn[df_trn['fecha_dato'] == '2016-06-28']
del df_trn

# 訓練データから新規購買件数だけを抽出
X = []
Y = []
for i, prod in enumerate(prods):
    prev = prod + '_prev'
    prX = trn[(trn[prod] == 1) & (trn[prev] == 0)]
    prY = np.zeros(prX.shape[0], dtype=np.int8) + i
    X.append(prX)
    Y.append(prY)
XY = pd.concat(X)
Y = np.hstack(Y)
XY['y'] = Y

# ここら辺は特徴量エンジニアリング
# 訓練、検証データに分離
vld_date = '2016-05-28'
XY_trn = XY[XY['fecha_dato'] != vld_date]
XY_vld = XY[XY['fecha_dato'] == vld_date]

#### モデル
- XGBoostモデルを使用
- よく使用するパラメータ
  - max_depth: ツリーモデルの最大の深さ。高ければ高いほど複雑なツリーモデルになり、過剰適合の原因になることがある。
  - eta: ディープラーニングのlearning rateのような概念。0と1の間の値を取る。
  - colsample_bytree: ツリーを生成するとき、訓練データから変数をサンプリングしてくれる比率。全てのツリーは全体の変数の一部だけを学習し、互いの弱点を補完し合う。普通0.6~0.9の値を使用する。
  - colsample_bylevel: ツリーのレベル別に訓練データの変数をサンプリングする比率。普通0.6~0.9の値を使用する。

- ただ、パラメータチューニングに時間を使うよりは特徴量エンジニアリングをした方が良いぜ

In [9]:
# XGBoostモデルのparameterを設定
param = {
    'booster': 'gbtree',
    'max_depth': 8,
    'nthread': 4,
    'num_class': len(prods),
    'objective': 'multi:softprob',
    'silent': 1,
    'eval_metric': 'mlogloss',
    'eta': 0.1,
    'min_child_weight': 10,
    'colsample_bytree': 0.8,
    'colsample_bylevel': 0.9,
    'seed': 2018,
}

In [9]:
# 訓練、検証データをXGBoost形式に変換
X_trn = XY_trn[features]
Y_trn = XY_trn['y']
dtrn = xgb.DMatrix(X_trn, label=Y_trn, feature_names=features)

X_vld = XY_vld[features]
Y_vld = XY_vld['y']
dvld = xgb.DMatrix(X_vld, label=Y_vld, feature_names=features)

# XGBoostモデルを訓練データで学習！！！！
watch_list = [(dtrn, 'train'), (dvld, 'eval')]
model = xgb.train(param, dtrn, num_boost_round=1000, evals=watch_list, early_stopping_rounds=20)

# 学習したモデルを保存
pickle.dump(model, open('./model/xgb.baseline.pkl', 'wb'))
best_ntree_limit = model.best_ntree_limit

[13:48:13] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "silent" } are not used.

[0]	train-mlogloss:2.72504	eval-mlogloss:2.73547
[1]	train-mlogloss:2.45700	eval-mlogloss:2.47071
[2]	train-mlogloss:2.27438	eval-mlogloss:2.28934


KeyboardInterrupt: 

In [10]:
# 保存したモデルをロード
with open('./model/xgb.baseline.pkl', 'rb') as f:
    model = pickle.load(f)
best_ntree_limit = model.best_ntree_limit

### 交差検証

In [10]:
from my_module import mapk

In [12]:
# MAP@7 評価基準のための準備作業
# 顧客識別番号の抽出
vld = trn[trn['fecha_dato'] == vld_date]
ncodpers_vld = vld['ncodpers'].values

# 検証データから新規購買を求める
for prod in prods:
    prev = prod + '_prev'
    padd = prod + '_add'
    vld[padd] = vld[prod] - vld[prev]
add_vld = vld[[prod+'_add' for prod in prods]].values
add_vld_list = [list() for i in range(len(ncodpers_vld))]

# 顧客別新規購買正答値を add_vld_listに保存し、総 countを count_vldに保存
count_vld = 0
for ncodper in range(len(ncodpers_vld)):
    for prod in range(len(prods)):
        if add_vld[ncodper][prod] > 0:
            add_vld_list[ncodper].append(prod)
            count_vld += 1

# 検証データから得ることのできる MAP@7 の最高点をあらかじめ求める (0.042663)
print(mapk.mapk(add_vld_list, add_vld_list, 7, 0.0))


C:\Users\mhira\AppData\Local\Temp\ipykernel_2504\3400318576.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vld[padd] = vld[prod] - vld[prev]
C:\Users\mhira\AppData\Local\Temp\ipykernel_2504\3400318576.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vld[padd] = vld[prod] - vld[prev]
C:\Users\mhira\AppData\Local\Temp\ipykernel_2504\3400318576.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

0.04266379915553903


In [14]:
# ダミー的にラベルを用意する
vld['y'] = 0

# 検証データに対する予測値を求める
X_vld = vld[features]
Y_vld = vld['y']
dvld = xgb.DMatrix(X_vld, label=Y_vld, feature_names=features)
preds_vld = model.predict(dvld, ntree_limit=best_ntree_limit)

# 前の月に保有していた商品は新規購買が不可能なので、確率値からあらかじめ１を引いておく
preds_vld = preds_vld - vld[[prod+'_prev' for prod in prods]].values

# 検証データの予測上位７個を抽出
result_vld = []
for ncodper, pred in zip(ncodpers_vld, preds_vld):
    y_prods = [(y, p, ip) for y, p, ip in zip(pred, prods, range(len(prods)))]
    y_prods = sorted(y_prods, key=lambda a: a[0], reverse=True)[:7]
    result_vld.append([ip for y, p, ip in y_prods])

# 検証データのMAP@7の点数を求める
print(mapk.mapk(add_vld_list, result_vld, 7, 0.0))

C:\Users\mhira\AppData\Local\Temp\ipykernel_2504\1895340737.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vld['y'] = 0
c:\Users\mhira\work\kaggle\kaggle_venv\lib\site-packages\xgboost\core.py:122: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


0.036472081566462485


### テストデータの予測及びKaggleへのアップロード

In [15]:
# XGBoostモデルを全体の訓練データで学習
X_all = XY[features]
Y_all = XY['y']
dall = xgb.DMatrix(X_all, label=Y_all, feature_names=features)
watch_list = [(dall, 'train')]
# ツリーの個数を増加したデータの量に比例して増やす
best_ntree_limit = int(best_ntree_limit * (len(XY_trn)+len(XY_vld))/len(XY_trn))
# XGBoostモデル再学習！
model = xgb.train(param, dall, num_boost_round=best_ntree_limit, evals=watch_list)
pickle.dump(model, open('./model/xgb.baseline.sumittion.pkl', 'wb'))

[13:50:44] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "silent" } are not used.

[0]	train-mlogloss:2.72661
[1]	train-mlogloss:2.45796
[2]	train-mlogloss:2.27490
[3]	train-mlogloss:2.13565
[4]	train-mlogloss:2.02164
[5]	train-mlogloss:1.92527
[6]	train-mlogloss:1.84695
[7]	train-mlogloss:1.77865
[8]	train-mlogloss:1.71683
[9]	train-mlogloss:1.66402
[10]	train-mlogloss:1.61725
[11]	train-mlogloss:1.57400
[12]	train-mlogloss:1.53480
[13]	train-mlogloss:1.50092
[14]	train-mlogloss:1.47010
[15]	train-mlogloss:1.44055
[16]	train-mlogloss:1.41443
[17]	train-mlogloss:1.39103
[18]	train-mlogloss:1.36930
[19]	train-mlogloss:1.34933
[20]	train-mlogloss:1.33099
[21]	train-mlogloss:1.31331
[22]	train-mlogloss:1.29760
[23]	train-mlogloss:1.28303
[24]	train-mlogloss:1.26968
[25]	train-mlogloss:1.25667
[26]	train-mlogloss:1.24477
[27]	train-mlogloss:1.23379
[28]	train-mlogloss:1.22361
[2

In [16]:
# 変数の重要度を出力
print("Feature importance:")
for kv in sorted([(k, v) for k, v in model.get_fscore().items()], key=lambda kv: kv[1], reverse=True):
    print(kv)

Feature importance:
('renta', 25131.0)
('age', 23997.0)
('antiguedad', 22604.0)
('antiguedad_prev', 16352.0)
('age_prev', 16143.0)
('fecha_alta_month', 15294.0)
('nomprov', 13979.0)
('fecha_alta_year', 11299.0)
('renta_prev', 11064.0)
('canal_entrada', 9446.0)
('nomprov_prev', 8048.0)
('canal_entrada_prev', 5980.0)
('fecha_alta_month_prev', 5407.0)
('fecha_alta_year_prev', 4136.0)
('sexo', 4110.0)
('ind_recibo_ult1_prev', 3975.0)
('ind_cco_fin_ult1_prev', 3505.0)
('ind_ecue_fin_ult1_prev', 3474.0)
('ind_cno_fin_ult1_prev', 3427.0)
('segmento', 2682.0)
('ind_reca_fin_ult1_prev', 2496.0)
('ind_tjcr_fin_ult1_prev', 2440.0)
('segmento_prev', 2303.0)
('tiprel_1mes', 1837.0)
('ind_nom_pens_ult1_prev', 1836.0)
('ind_valo_fin_ult1_prev', 1834.0)
('ind_ctop_fin_ult1_prev', 1793.0)
('ind_dela_fin_ult1_prev', 1754.0)
('sexo_prev', 1566.0)
('ind_nomina_ult1_prev', 1547.0)
('ind_actividad_cliente', 1412.0)
('ind_ctpp_fin_ult1_prev', 1373.0)
('tiprel_1mes_prev', 1362.0)
('ind_fond_fin_ult1_prev', 11

In [13]:
# 保存したモデルをロード
with open('./model/xgb.baseline.submission.pkl', 'rb') as f:
    model = pickle.load(f)
f.close()
best_ntree_limit = model.best_ntree_limit

In [29]:
# 提出用にテストデータの予測値を求める
X_tst = tst[features]
dtst = xgb.DMatrix(X_tst, feature_names=features)
preds_tst = model.predict(dtst, ntree_limit=best_ntree_limit)
ncodpers_tst = tst['ncodpers'].values
preds_tst = (preds_tst-tst[[prod+'_prev' for prod in prods]]).values

c:\Users\mhira\work\kaggle\kaggle_venv\lib\site-packages\xgboost\core.py:122: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


In [30]:
# 提出ファイルを生成
with open('./output/xgb_baseline_1.csv', 'w') as f:
    f.write('ncodpers,added_products\n')
f.close()
with open('./output/xgb_baseline_1.csv', 'a') as f:
    for ncodper, pred in zip(ncodpers_tst, preds_tst):
        y_prods = [(y, p, ip) for y, p, ip in zip(pred, prods, range(len(prods)))]
        y_prods = sorted(y_prods, key=lambda a: a[0], reverse=True)[:7]
        y_prods = [p for y, p, ip in y_prods]
        f.write('{},{}\n'.format(int(ncodper), ' '.join(y_prods)))
f.close()